In [2]:
from datetime import datetime
from winq import *
import hiq_pyfetch as fetch
from winq.selector import *
db = default(log_level='info')

# test_end_date = '20230322'
# with_rt_kdata = False
run_task_count = 50

test_end_date = '20230402'
with_rt_kdata = False


In [3]:
# 选择热门股票，热门股票必定属于某个热门的概念
strategy = strategies['test.RightSide'](db=db, 
                                        test_end_date=test_end_date, with_rt_kdata = with_rt_kdata,
                                        run_task_count=run_task_count,
                                        load_daily=db.load_stock_concept_daily, 
                                        load_info=db.load_stock_concept,
                                        fetch_daily=fetch.fetch_stock_concept_daily
                                       )
await strategy.prepare(min_hit_days=2,
                       max_down_in_rise=0.5,
                       max_up_in_rise=10.0,
                       max_falling_down=1.0,
                       max_turnover=10.0,
                       min_high_ndays=2,
                       test_recent_ndays=2,
                       test_recent_ndays_up=2.0,
                       sort_by='hit_chg_pct')

data = await strategy.run()
# 观察热门概念，可能需要微调参数

if data is not None and  len(data) > 0:
    data = data[~data['name'].str.contains('昨日')].reset_index(drop=True)
    test_end_date = strategy.test_end_date.strftime('%Y%m%d')
    export_codes_to_path(path='/Users/luoguochun/Downloads/stock/analyze/hot_concept_{}.txt'.format(test_end_date), data=data)
data

处理 BK1127(AI芯片): 100%|██████████| 418/418 [00:09<00:00, 45.96it/s]        


,code,name,nday_close,close,nday_chg_pct,hit_start,hit_chg_pct,hit_days
0,BK1063,重组蛋白,758.919983,789.429993,4.02,2023-03-27,4.02,2
1,BK0887,鸡肉概念,929.989990,964.169983,3.68,2023-03-27,3.68,2
2,BK0882,猪肉概念,1481.650024,1519.660034,2.57,2023-03-27,2.56,2


In [4]:
data_detail = await db.load_stock_concept_detail(filter={'code': {'$in': list(data['code'].values)}})
codes = list(set(data_detail['stock_code'].values))
len(codes)

58

In [5]:
strategy = strategies['test.RightSide'](db=db, 
                                        test_end_date=test_end_date,
                                        with_rt_kdata = with_rt_kdata,
                                        run_task_count=run_task_count,
                                        load_daily=db.load_stock_daily, 
                                        load_info=db.load_stock_info,
                                        fetch_daily=fetch.fetch_stock_bar
                                       )
await strategy.prepare(min_hit_days=1,
                       max_down_in_rise=3.0,
                       max_up_in_rise=20.0,
                       max_falling_down=2.0,
                       max_turnover=10.0,
                       min_high_ndays=2,
                       test_recent_ndays=1,
                       test_recent_ndays_up=0.0,
                       sort_by='hit_chg_pct')

data = await strategy.run(codes=codes)
if data is not None and len(data) > 0:
    test_end_date = strategy.test_end_date.strftime('%Y%m%d')
    export_codes_to_path(path='/Users/luoguochun/Downloads/stock/analyze/hot_stock_{}.txt'.format(test_end_date), data=data)

data

处理 sz301116(益客食品): 100%|██████████| 57/57 [00:08<00:00,  6.51it/s]  


,code,name,nday_close,close,nday_chg_pct,hit_start,hit_chg_pct,hit_days
0,sz000048,京基智农,20.510000,21.280001,3.75,2023-03-27,7.53,2
1,sz300761,立华股份,37.619999,40.049999,6.46,2023-03-28,6.46,1
2,sz000860,顺鑫农业,31.270000,33.270000,6.40,2023-03-28,6.40,1
3,sz002299,圣农发展,23.510000,24.959999,6.17,2023-03-28,6.17,1


In [2]:
# 热门股票超跌
strategy = strategies['test.SuperFalling'](db=db, 
                                        test_end_date=test_end_date, with_rt_kdata = with_rt_kdata,
                                        run_task_count=run_task_count,
                                        load_daily=db.load_stock_daily, 
                                        load_info=db.load_stock_info,
                                        fetch_daily=fetch.fetch_stock_bar
                                       )
await strategy.prepare(is_trading=False,
                       max_first_day_rise=3.0,
                        max_down_as_rise=-1.5,
                        max_meet_high_days=5,
                        min_high_falling=2.0,
                        max_high_falling=10.0,
                        min_rise_to_high_ndays=10.0,
                        sort_by='chg_pct')

data = await strategy.run()

if data is not None and  len(data) > 0:
    test_end_date = strategy.test_end_date.strftime('%Y%m%d')
#    export_codes_to_path(path='/Users/luoguochun/Downloads/stock/analyze/supper_falling_{}.txt'.format(test_end_date), data=data)

data

处理 sh605168(三人行): 100%|██████████| 4947/4947 [01:52<00:00, 44.14it/s]     


,code,name,chg_pct,high_falling_days,high_falling_down,high_rise_days,high_rise
0,sz300921,南凌科技,2.97,3,-7.65,7,10.239197
1,sz300063,天龙集团,2.96,4,-3.83,13,12.980769
2,sh600797,浙大网新,2.91,4,-6.65,13,21.617855
3,sz000676,智度股份,2.90,4,-7.16,8,21.404686
4,sh688393,安必平,2.87,4,-4.41,4,14.006765
5,sh603603,*ST博天,2.74,2,-2.44,6,17.333330
6,sz002123,梦网科技,2.64,4,-9.89,4,11.436537
7,sz002373,千方科技,2.59,4,-8.00,4,18.093697
8,sz003016,欣贺股份,2.53,3,-2.10,3,13.865925
9,sh601900,南方传媒,2.52,3,-2.01,20,17.176471


In [7]:
test_end_date = '20230331'
with_rt_kdata = False

strategy = strategies['test.SuperFalling'](db=db, 
                                        test_end_date=test_end_date, with_rt_kdata = with_rt_kdata,
                                        run_task_count=run_task_count,
                                        load_daily=db.load_stock_daily, 
                                        load_info=db.load_stock_info,
                                        fetch_daily=fetch.fetch_stock_bar
                                       )
await strategy.prepare(max_first_day_rise=3.0,
                       max_first_day_down=-6.0,
                        max_down_as_rise=-1.5,
                        max_meet_high_days=10,
                        min_high_falling=2.0,
                        max_high_falling=10.0,
                        min_rise_to_high_ndays=10.0,
                        sort_by='chg_pct')

data = await strategy.test('sh688619')
data